In [86]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sys, os
from scipy.stats import poisson,skellam
from scipy.optimize import minimize

sys.path.insert(0, '/home/igormago/projects/DeepFootball')

from conf.configparser import conf
datasets_path = conf['path']['datasets']

plt.style.use('ggplot')

import statsmodels.api as sm
import statsmodels.formula.api as smf

### Reading Data

In [120]:
df_file = os.path.join(datasets_path,'be_matches','Brazil A-dcw.csv')
df = pd.read_csv(df_file, low_memory=False)

In [121]:
df['ryes'] = df['res_bts'] == 'y'
df['rno'] = df['res_bts'] == 'n'
df['res_cat'] = df['res_bts'] == 'y'
df['pred_cat'] = df['pyes'] > df['pno']

In [124]:
from sklearn.metrics import accuracy_score
accuracy_score(df['res_cat'], df['pred_cat'])

0.4676470588235294

In [125]:
from sklearn.metrics import brier_score_loss
brier_score_loss(df['ryes'], df['pyes'])

0.29527299323282175

In [97]:
dtrain = df[(df.ss_yi >= 2013) & (df.ss_yi <= 2016)]
dtest = df[(df.ss_yi >= 2017) & (df.ss_yi <= 2018)]

In [100]:
def simulate_match(foot_model, homeTeam, awayTeam, max_goals=10):
    home_goals_avg = foot_model.predict(pd.DataFrame(data={'th': homeTeam, 
                                                            'ta': awayTeam,'home':1},
                                                      index=[1])).values[0]
    away_goals_avg = foot_model.predict(pd.DataFrame(data={'th': awayTeam, 
                                                            'ta': homeTeam,'home':0},
                                                      index=[1])).values[0]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [home_goals_avg, away_goals_avg]]
    
    matrix = np.outer(np.array(team_pred[0]), np.array(team_pred[1]))
    btts_yes = matrix[0,0] + matrix[0,1] + matrix[1,0]
    btts_no = 1 - btts_yes
    
    return btts_yes, btts_no

In [114]:
dtrain_total = dtrain

for year in [2017,2018]:
    dround_pred = dtest[(dtest.ss_yi == year) & (dtest.rg < 5)]
    dtrain_total = pd.concat([dtrain_total,dround_pred], sort=True)
    
    for nround in range (5,39):
        dround_pred = dtest[(dtest.ss_yi == year) & (dtest.rg == nround)]
                
        dtrain_total['days_dif'] = (max(dtrain_total['dt']) - dtrain_total['dt']).dt.days
        dtrain_trans = pd.concat([dtrain_total[['th','ta','gh']].assign(home=1).rename(columns={'gh':'goals'}), 
                             dtrain_total[['ta','th','ga']].assign(home=0).rename(columns={'ga':'goals'})], sort=True)
        
        model = smf.glm(formula="goals ~ home + th + ta", data=dtrain_trans, 
                        family=sm.families.Poisson()).fit()
        
        for m in dround_pred.iterrows():
            th, ta = m[1]['th'], m[1]['ta']
            pyes, pno = simulate_match(model, th, ta, 10)
            dtest.loc[m[0],'pyes'] = pyes
            dtest.loc[m[0],'pno'] = pno
            
        dround_pred.loc[:,'pyes'] = lyes
        dround_pred.loc[:,'no'] = lno
            
        dtrain_total = pd.concat([dtrain_total,dround_pred], sort=True)
            

In [ ]:
def rho_correction(x, y, lambda_x, mu_y, rho):
    if x==0 and y==0:
        return 1- (lambda_x * mu_y * rho)
    elif x==0 and y==1:
        return 1 + (lambda_x * rho)
    elif x==1 and y==0:
        return 1 + (mu_y * rho)
    elif x==1 and y==1:
        return 1 - rho
    else:
        return 1.0


In [140]:
def solve_parameters(dataset, debug = False, init_vals=None, options={'disp': True, 'maxiter':5},
                     constraints = [{'type':'eq', 'fun': lambda x: sum(x[:20])-20}] , **kwargs):
    teams = np.sort(dataset['th'].unique())
    # check for no weirdness in dataset
    away_teams = np.sort(dataset['ta'].unique())
    if not np.array_equal(teams, away_teams):
        raise ValueError("Something's not right")
    n_teams = len(teams)
    if init_vals is None:
        # random initialisation of model parameters
        init_vals = np.concatenate((np.random.uniform(0,1,(n_teams)), # attack strength
                                      np.random.uniform(0,-1,(n_teams)), # defence strength
                                      np.array([0, 1.0]) # rho (score correction), gamma (home advantage)
                                     ))
    def dc_log_like(x, y, alpha_x, beta_x, alpha_y, beta_y, rho, gamma):
        lambda_x, mu_y = np.exp(alpha_x + beta_y + gamma), np.exp(alpha_y + beta_x) 
        return (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) + 
                np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))

    def estimate_paramters(params):
        score_coefs = dict(zip(teams, params[:n_teams]))
        defend_coefs = dict(zip(teams, params[n_teams:(2*n_teams)]))
        print(len(dataset))
        rho, gamma = params[-2:]
        log_like = [dc_log_like(row.gh, row.ga, score_coefs[row.th], defend_coefs[row.th],
                     score_coefs[row.ta], defend_coefs[row.ta], rho, gamma) for row in dataset.itertuples()]
        est = -sum(log_like)
        print(len(log_like))
        return est
    opt_output = minimize(estimate_paramters, init_vals, options=options,
                          constraints = constraints, **kwargs)
    if debug:
        # sort of hacky way to investigate the output of the optimisation process
        return opt_output
    else:
        return dict(zip(["attack_"+team for team in teams] + 
                        ["defence_"+team for team in teams] +
                        ['rho', 'home_adv'],
                        opt_output.x))

In [141]:
params = solve_parameters(dtrain)

1520
1520
1520
1520
1520
1520
1520
1520
1520
1520
1520
1520
1520
1520
1520
1520
1520
1520
1520
1520
1520


KeyboardInterrupt: 

In [ ]:
print(params)